### First Order Feedback Filter

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

frameSize = 2048

# x: input signal (impulse)
# a0: filter coefficient (forward path)
# b1: filter coefficient (backward path)
def feedbackFilter(x, a0, b1, delay):
    y = np.zeros(x.size)
    for n in range(0, x.size):
        y[n] = a0 * x[n] - b1 * y[n-delay]
    return y

# impulse signal
impulse = np.zeros(frameSize)
impulse[0] = 1

@widgets.interact(b1=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.8))
def execute(b1=0.8):

    # get the impulse response of the filter
    impulseResponse = feedbackFilter(impulse, 1.0, b1, 1)

    # setup the plot
    _, axes = plt.subplots(1, 2, figsize=(16,6))

    # plot the impulse signal
    ax = axes[0]
    ax.plot(impulse, 'bo-')
    ax.set_xlabel('Sample count')
    ax.set_ylabel('Amplitude')
    ax.set_ylim(-0.1, 1.1)
    ax.set_xlim(-0.5, 10)
    ax.set_title("Impulse")
    ax.grid(which='both', axis='both')

    # plot the impulse response
    ax = axes[1]
    ax.plot(impulseResponse, 'ro-')
    ax.set_xlabel('Sample count')
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlim(-0.5, 50)
    ax.set_title("Impulse Response")
    ax.grid(which='both', axis='both')

    plt.show

interactive(children=(FloatSlider(value=0.8, description='b1', max=1.0, min=-1.0, step=0.01), Output()), _dom_…

We can then perform a Fourier transform on the impulse signal. This will give us an array of complex numbers which is stored in spectrum. Extracting the magnitudes of the complex numbers and plotting them will give us a frequency response plot, and extracting the angles will give us a phase response plot

In [2]:
from scipy.fftpack import fft

@widgets.interact(b1=widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, value=0.8))
def execute(b1=0.8):

    # fft returns an array of complex numbers representing the frequency components of the input signal, more info: https://docs.scipy.org/doc/scipy/tutorial/fft.html
    spectrum = fft(feedbackFilter(impulse, 1.0, b1, 1))

    # get the frequency components
    x = np.linspace(0, 0.5, spectrum.size//2)

    # setup the plot
    fig, axes = plt.subplots(1, 2, figsize=(16,6))

    # get the amplitude of the frequency components
    with np.errstate(divide='ignore'):
        y = (20 * np.log10(np.abs(spectrum)))[:spectrum.size//2]

    # plot the frequency response
    ax = axes[0]
    ax.plot(x, y, 'b')
    #ax.set_xscale('log')
    ax.set_xlabel('Normalized frequency')
    ax.set_ylabel('Amplitude (dB)')
    ax.set_title("Frequency Response")
    ax.grid(which='both', axis='both')

    # get the phase of the frequency components
    y = np.degrees(np.angle(spectrum))[:spectrum.size//2]

    # plot the phase response
    ax = axes[1]
    ax.plot(x, y, 'r')
    # ax.set_xscale('log')
    ax.set_xlabel('Normalized frequency')
    ax.set_ylabel('Phase (deg)')
    ax.set_title("Phase Response")
    ax.grid(which='both', axis='both')

    plt.show()

interactive(children=(FloatSlider(value=0.8, description='b1', max=1.0, min=-1.0, step=0.001), Output()), _dom…